# 箱体和其他零件的尺寸设计

我们现在已经得到了两级减速齿轮的所有参数，以及用工作扭矩计算的轴颈，我们把它封装到`design.design_all`里，可以通过输出牵引力、输送带速度、滚筒直径直接计算

In [ ]:
from design import design_all
import numpy as np
import matplotlib.pyplot as plt
import json

F_w = 2.3 #v kN
v_w = 0.95 #m/s
D = 440 #mm

out1, out2, out_shaft = design_all(F_w, v_w, D)
z1_small,z2_big,m12,b1_small,b2_big,a12 = out1
z2_small,z3_big,m23,b2_small,b3_big,a23 = out2
d1,d2,d3 = out_shaft[0]

d1_small = z1_small * m12
d2_big = z2_big * m12
d2_small = z2_small * m23
d3_big = z3_big * m23


工作转速：41.236 rpm
总效率为: 0.808
电机所需功率为：2703.444 W
总传动比为:35.285
i1 = 6.773, i2 = 5.210
轴1的转速为:1455.000, 轴2的转速为:214.831, 轴3的转速为:41.236
P1 = 2119.887 W, P2 = 2035.939 W, P3 = 1955.316 W
d1 = 11.677 mm, d2 = 21.797 mm, d3 = 37.281 mm
T1 = 13.914 N*m, T2 = 90.505 N*m, T3 = 452.843 N*m
epsilon_a = 1.691
试算 d_1t = 32.64 mm
实际 d_1H = 34.47 mm
试算 m_t = 1.26 mm
K_Fb = 1.15
实际 m_F = 1.30 mm
实际 d_1F = 22.09 mm
齿面接触疲劳强度计算: m_H = 2.028, d_H = 34.469
齿根弯曲疲劳强度计算: m_F = 1.300, d_F = 22.094

    主要设计结论:
    齿数z1 = 17
    齿数z2 = 115
    模数m = 2 mm
    压力角alpha = 20°
    中心距a = 132 mm
    齿宽b1 = 32 mm
    齿宽b2 = 27 mm
    小齿轮选用40Cr(调质),
    大齿轮选用45钢(调质),
    7级精度
    
epsilon_a = 1.707
试算 d_1t = 59.57 mm
实际 d_1H = 62.04 mm
试算 m_t = 2.09 mm
K_Fb = 1.15
实际 m_F = 2.01 mm
实际 d_1F = 40.29 mm
齿面接触疲劳强度计算: m_H = 3.102, d_H = 62.043
齿根弯曲疲劳强度计算: m_F = 2.014, d_F = 40.288

    主要设计结论:
    齿数z1 = 31
    齿数z2 = 162
    模数m = 2 mm
    压力角alpha = 20°
    中心距a = 193 mm
    齿宽b1 = 54 mm
    齿宽b2 = 49 mm
    小齿轮选用40Cr(调质),
  

In [3]:
print('齿数: z1_small = {:.3f}, z2_big = {:.3f}, z2_small = {:.3f}, z3_big = {:.3f}'.format(z1_small,z2_big,z2_small,z3_big))
print('中心距 a12={:.3f} a23={:.3f}'.format(a12,a23))
print('齿宽 b1_small={:.3f} b2_big={:.3f} b2_small={:.3f} b3_big={:.3f}'.format(b1_small,b2_big,b2_small,b3_big))
print('轴颈下界: d1={:.3f} d2={:.3f} d3={:.3f}'.format(d1,d2,d3))
print('分度圆直径 d1_small={:.3f} d2_big={:.3f} d2_small={:.3f} d3_big={:.3f}'.format(d1_small,d2_big,d2_small,d3_big))

齿数: z1_small = 17.000, z2_big = 115.000, z2_small = 31.000, z3_big = 162.000
中心距 a12=132.000 a23=193.000
齿宽 b1_small=32.200 b2_big=27.200 b2_small=54.600 b3_big=49.600
轴颈下界: d1=11.677 d2=21.797 d3=37.281
分度圆直径 d1_small=34.000 d2_big=230.000 d2_small=62.000 d3_big=324.000


现在我们来设计箱体

## 设计螺栓

根据机械设计`课程设计 Page 23`我们发现每一个螺栓拥有三个参数

* $c_1$: 至外箱壁距离
* $c_2$: 至凸边缘的距离
* $D_0$: 沉头座坑直径
* $d$: 螺栓直径

因此，我们做了以下的事情
* 建立一个类`Bolt`用于表示螺栓
* 录制`课程设计 Page 23 表 5-3`为一个字典`Bolt_to_c1c2D0`
* 定义了一个函数`select_bolt(d)`，给定螺栓计算直径，选择合适的螺栓直径

In [ ]:
ALL_BOLT = [8,10,12,16,20,24,30]
def select_bolt(d):
    k = ALL_BOLT
    k = np.array(k)
    d = np.abs(k - d)
    idx = np.argmin(d)
    return ALL_BOLT[idx]

Bolt_to_c1c2D0 = {
    8: (13,11,18),
    10: (16,14,22),
    12: (18,16,26),
    16: (22,20,33),
    20: (26,24,20),
    24: (34,28,48),
    30: (40,34,61)
}


class Bolt:
    def __init__(self, d):
        self.d = d
        self.d = select_bolt(self.d)
        self.c1, self.c2, self.D0 = Bolt_to_c1c2D0[self.d]
    def __dict__(self):
        return {'d':self.d, 'c1':self.c1, 'c2':self.c2, 'D0':self.D0}
    


接下来，我们创建了一个字典`box`用于储存箱体的设计尺寸，根据`课程设计 Page 23 表5-2`进行设计各个尺寸。

In [5]:
box = {}

首先计算箱座壁厚，由`表5-1`知，对于二级圆柱齿轮减速器，由第二齿轮副的中心距决定

$$\delta = \max \{0.025a + 1, 8 \}$$

In [6]:
box['delta'] = max(8, 0.025*a23+3)
print('delta = {:.3f} mm'.format(box['delta']))

delta = 8.000 mm


箱盖壁厚：
$$ \delta_1 = \max \{ 0.85 \delta, 8 \} $$


In [7]:
box['delta_1'] = max(0.85*box['delta'], 8)
print('delta_1 = {:.3f} mm'.format(box['delta_1']))

delta_1 = 8.000 mm


地脚螺栓直径：
$$ d_f = 0.036a + 12 $$
传入`Bolt`中自动化整选型

In [ ]:
box['d_f'] = Bolt(0.036 * a23 + 12)